<a href="https://colab.research.google.com/github/jpvmm/FASAM_NLP/blob/master/HandsOn1_NLP_Cl%C3%A1ssico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Começando os trabalhos

import pandas as pd

df_train = pd.read_csv('https://raw.githubusercontent.com/jpvmm/FASAM_NLP/master/train_sample.csv')
df_train.head()

In [0]:
df_test = pd.read_csv('https://raw.githubusercontent.com/jpvmm/FASAM_NLP/master/test_sample.csv')
df_test.head()

# **Análise Exploratória de Dados**


---
Vamos dar uma olhada o que tem nesse dataset

* Quantos pontos de dados esse dataset possui?
* Quantas categorias diferentes os títulos possuem?
* Quantos pontos de dados temos por língua?
* Quantos pontos de dados por label quality?

In [0]:
#Pontos de dados
df_train['category'].count()

In [0]:
#Quantas categorias únicas temos?
df_train['category'].nunique()

In [0]:
import seaborn as sns
sns.countplot(df_train['language'])

In [0]:
sns.countplot(df_train['label_quality'])

## Conclusões de EDA:
* Duas linguagens!
* Dataset bem distribuído entre as linguagens;
* Grande quantidade de labels;

# **Começando os trabalhos**


---
**Pergunta**: Como podemos trabalhar com duas linguages diferentes? Qual estratégia poderia ser adotada?


In [0]:
df_port = df_train[df_train['language']=='portuguese']
df_spani = df_train[df_train['language']=='spanish']

In [0]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

In [0]:
#Alguns exemplos de stopwords
stops = set(stopwords.words('portuguese'))

stops

In [0]:
#Função pra limpeza de documentos
import re
import string
def limpaTexto(texto):

  tokens = texto.split()

  #Regex para filtro de caracteres
  re_puc = re.compile('[%s]' % re.escape(string.punctuation))
  #Remoção de pontuação
  tokens = [re_puc.sub('', w ) for w in tokens]
  #Remoção de tokens não alfabéticos
  tokens = [word for word in tokens if word.isalpha()]
  #Remoção de stopwords
  stop_words = set(stopwords.words('portuguese'))

  tokens = [w for w in tokens if not w in stop_words]
  tokens = [word for word in tokens if len(word) > 1]
  tokens = ' '.join(tokens)

  return tokens


In [0]:
df_port['title'] = df_port['title'].apply(limpaTexto)

df_port.head()

In [0]:
y = df_port['category'].values
x = df_port['title']

In [0]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.1)

In [0]:
clf = Pipeline([('tfidf', TfidfVectorizer(lowercase=True, ngram_range=(1,1))),
                ('clf', LinearSVC())])

In [0]:
clf.fit(x_train, y_train)

In [0]:
y_pred = clf.predict(x_test)
print(classification_report(y_test, y_pred))

## Modelo para Espanhol

In [0]:
stops_spani = set(stopwords.words('spanish'))
stops_spani

In [0]:
import re
import string
def limpaTextoSpani(texto):

  tokens = texto.split()

  #Regex para filtro de caracteres
  re_puc = re.compile('[%s]' % re.escape(string.punctuation))
  #Remoção de pontuação
  tokens = [re_puc.sub('', w ) for w in tokens]
  #Remoção de tokens não alfabéticos
  tokens = [word for word in tokens if word.isalpha()]
  #Remoção de stopwords
  stop_words = set(stopwords.words('spanish'))

  tokens = [w for w in tokens if not w in stop_words]
  tokens = [word for word in tokens if len(word) > 1]
  tokens = ' '.join(tokens)

  return tokens

df_spani['title'] = df_spani['title'].apply(limpaTextoSpani)

df_spani.head()

In [0]:
ys = df_spani['category'].values
xs = df_spani['title']

x_train, x_test, y_train, y_test = train_test_split(xs,ys, test_size = 0.1)

In [0]:
clf2 = Pipeline([('tfidf', TfidfVectorizer(lowercase=True, ngram_range=(1,1))),
                ('clf', LinearSVC())])

clf2.fit(x_train, y_train)

In [0]:
y_pred = clf2.predict(x_test)
print(classification_report(y_test, y_pred))

In [0]:
accuracy_score(y_test, y_pred)